# Agent Step-wise Execution

This notebook shows you how to run step-wise execution and full-execution with agents.

In [ ]:
!pip install llama-index

In [ ]:
import json
from typing import Sequence, List

from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

tools = [multiply_tool, add_tool]

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo")

## Test OpenAI Agent

In [ ]:
from llama_index.agent.executor.base import AgentEngine
from llama_index.agent.openai.step import OpenAIAgentStepEngine
from llama_index.agent.legacy.openai_agent import OpenAIAgent as OldOpenAIAgent

openai_step_engine = OpenAIAgentStepEngine.from_tools(
    tools, llm=llm, verbose=True
)
agent = AgentEngine(openai_step_engine)

### Test Chat

This will iterate step-wise until the agent is done with the current task.

In [ ]:
agent.chat("Hi")

AgentChatResponse(response='Hello! How can I assist you today?', sources=[], source_nodes=[])

In [ ]:
response = agent.chat("What is (121 * 3) + 42?")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363

=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405



In [ ]:
response

AgentChatResponse(response='The result of (121 * 3) + 42 is 405.', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[])

In [ ]:
response = agent.stream_chat("What is (121 * 3) + 42?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363

=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405




In [ ]:
str(response)

'The result of (121 * 3) + 42 is 405.'

### Test Step-Wise Execution

We can also break this down into steps.

In [ ]:
# start task
task = agent.create_task("What is (121 * 3) + 42?")

In [ ]:
step_output = agent.run_step(task.task_id)

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363



In [ ]:
step_output.output

AgentChatResponse(response='None', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363)], source_nodes=[])

In [ ]:
step_output = agent.run_step(task.task_id)

=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405



In [ ]:
step_output.output

AgentChatResponse(response='None', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[])

In [ ]:
step_output = agent.run_step(task.task_id)

In [ ]:
# display final response
print(step_output.is_last)
step_output.output.response

True


'The result of (121 * 3) + 42 is 405.'

### Test Wrapper OpenAIAgent

The wrapper OpenAIAgent class now wraps the `AgentEngine` with the step-wise executor.

In [ ]:
# test wrapper openai agent
from llama_index.agent import OpenAIAgent

openai_agent_w = OpenAIAgent.from_tools(tools, llm=llm, verbose=True)

In [ ]:
response = openai_agent_w.chat("What is (121 * 3) + 42?")

=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 3}
Got output: 363

=== Calling Function ===
Calling function: add with args: {"a": 42, "b": 0}
Got output: 42

=== Calling Function ===
Calling function: add with args: {"a":363,"b":42}
Got output: 405



## Test ReAct Agent

We do the same experiments, but with ReAct.

In [ ]:
llm = OpenAI(model="gpt-4-1106-preview")

In [ ]:
from llama_index.agent.executor.base import AgentEngine
from llama_index.agent.react.step import ReActAgentStepEngine

In [ ]:
react_step_engine = ReActAgentStepEngine.from_tools(
    tools, llm=llm, verbose=True
)
agent = AgentEngine(react_step_engine)

In [ ]:
agent.chat("Hi")

Thought: (Implicit) I can answer without any more tools!
Response: Hello! How can I assist you today?


AgentChatResponse(response='Hello! How can I assist you today?', sources=[], source_nodes=[])

In [ ]:
response = agent.chat("What is (121 * 3) + 42?")

Thought: I need to use a tool to help me calculate the multiplication part of the question first.
Action: multiply
Action Input: {'a': 121, 'b': 3}
Observation: 363
Thought: Now that I have the result of the multiplication, I need to add 42 to it.
Action: add
Action Input: {'a': 363, 'b': 42}
Observation: 405
Thought: I can answer without using any more tools.
Response: The result of (121 * 3) + 42 is 405.


In [ ]:
response

AgentChatResponse(response='The result of (121 * 3) + 42 is 405.', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[])

In [ ]:
# start task
task = agent.create_task("What is (121 * 3) + 42?")

In [ ]:
step_output = agent.run_step(task.task_id)

Thought: I need to use the multiply tool to calculate 121 * 3 first, then use the add tool to add 42 to the result.
Action: multiply
Action Input: {'a': 121, 'b': 3}
Observation: 363


In [ ]:
step_output.output

AgentChatResponse(response='Observation: 363', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363)], source_nodes=[])

In [ ]:
step_output = agent.run_step(task.task_id)

Thought: Now that I have the result of 121 * 3, which is 363, I need to add 42 to it using the add tool.
Action: add
Action Input: {'a': 363, 'b': 42}
Observation: 405


In [ ]:
step_output.output

AgentChatResponse(response='Observation: 405', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[])

In [ ]:
step_output = agent.run_step(task.task_id)

Thought: I can answer without using any more tools.
Response: The result of (121 * 3) + 42 is 405.


In [ ]:
step_output.output

AgentChatResponse(response='The result of (121 * 3) + 42 is 405.', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[])

In [ ]:
# test wrapper react agent
from llama_index.agent import ReActAgent

react_agent_w = ReActAgent.from_tools(tools, llm=llm, verbose=True)

In [ ]:
response = react_agent_w.chat("What is (121 * 3) + 42?")

Thought: I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 121, 'b': 3}
Observation: 363
Thought: Now that I have the result of the multiplication, I need to add 42 to it.
Action: add
Action Input: {'a': 363, 'b': 42}
Observation: 405
Thought: I can answer without using any more tools.
Response: (121 * 3) + 42 is 405.
